In [9]:
import torch
import torch.nn as nn
import pandas as pd
import torch.nn.functional as F

import numpy as np

import data_process as dp

from pointnet_model import get_model

In [10]:
# 读取训练数据
train_data = pd.read_csv('world_data.csv')

# 查看最后一列
print(train_data.iloc[:, -1].value_counts())

1    1014
0    1013
3    1010
2    1009
Name: 3, dtype: int64


In [11]:
#随机抽样30%的数据作为测试集
test_data = train_data.sample(frac=0.2)
train_data = train_data.drop(test_data.index)

print(train_data.shape, test_data.shape)

(3237, 64) (809, 64)


In [12]:
X, y = dp.data_to_points_cloud(train_data)
Xval, yval = dp.data_to_points_cloud(test_data)

print(X.shape, y.shape, Xval.shape, yval.shape)




torch.Size([3237, 3, 21]) torch.Size([3237, 4]) torch.Size([809, 3, 21]) torch.Size([809, 4])


In [13]:
model = get_model(num_classes=4, global_feat=True, feature_transform=False, channel=3)

# 定义损失函数和优化器
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.00004)


In [14]:
num_epochs = 20
num_samples = X.shape[0]
batch_size = 32


for epoch in range(num_epochs):
    train_loss = 0.0
    for i in range(0, num_samples, batch_size):
        input = X[i:i+batch_size]
        label = y[i:i+batch_size]


 
        # 前向传播
        model.train()
        output, trans, trans_feat = model(input)

        l = loss(output, label)
        
        # 反向传播和优化
        optimizer.zero_grad()
        l.backward()
        optimizer.step()

        train_loss += l.item()

    train_loss /= num_samples
    with torch.no_grad():
        model.eval()
        y_pred, _, _ = model(Xval)
        val_loss = loss(y_pred, yval).item()

    # 打印每个 epoch 的损失
    print(f'Epoch {epoch+1}/{num_epochs}, train_Loss: {train_loss}, val_Loss: {val_loss}')
    
    if val_loss < 0.005:
        break
    


    # 保存模型
torch.save(model.state_dict(), 'model.pth')

Epoch 1/30, train_Loss: 0.012404730129764887, val_Loss: 0.18454615771770477
Epoch 2/30, train_Loss: 0.008041644675480498, val_Loss: 0.1198369562625885
Epoch 3/30, train_Loss: 0.006207784230984542, val_Loss: 0.08065489679574966
Epoch 4/30, train_Loss: 0.005068818769559545, val_Loss: 0.05702415108680725
Epoch 5/30, train_Loss: 0.004376450421764485, val_Loss: 0.046651244163513184
Epoch 6/30, train_Loss: 0.003926509247265887, val_Loss: 0.037328965961933136
Epoch 7/30, train_Loss: 0.0036968308920141426, val_Loss: 0.033043473958969116
Epoch 8/30, train_Loss: 0.0034013491102048636, val_Loss: 0.03333039954304695
Epoch 9/30, train_Loss: 0.003089267563720452, val_Loss: 0.027341946959495544
Epoch 10/30, train_Loss: 0.002909382889815455, val_Loss: 0.0245471503585577
Epoch 11/30, train_Loss: 0.0026937545768302056, val_Loss: 0.022624576464295387
Epoch 12/30, train_Loss: 0.002563684423801374, val_Loss: 0.020247384905815125
Epoch 13/30, train_Loss: 0.0024421987436529955, val_Loss: 0.018633464351296425

In [ ]:
with torch.no_grad():
    model.eval()
    pre_val, _, _ = model(Xval)

head = 10

print(pre_val.shape)
print(pre_val[:head])
print(yval[:head])

torch.Size([809, 4])
tensor([[ 0.0054, -0.0673, -0.1432,  1.1049],
        [ 1.0114, -0.0779, -0.0182,  0.0222],
        [ 0.0465, -0.0415,  0.8725,  0.0449],
        [-0.0584, -0.0654, -0.1175,  1.1001],
        [ 0.0089, -0.1185,  0.0111,  0.9493],
        [ 0.9379, -0.0360,  0.0170, -0.0287],
        [-0.0369, -0.0714, -0.0166,  1.0312],
        [-0.0136, -0.0846, -0.0328,  1.0173],
        [ 0.0259,  0.5445,  0.0208, -0.0309],
        [-0.0390, -0.0828, -0.0490,  1.0712]])
tensor([[0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.]])


In [ ]:
x = Xval[0]
x = x.reshape(1, x.shape[0], x.shape[1])